In [8]:
# import libraries
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

# read in data
required_cols = ['JobID','Partition', 'Account', 'Group', 'GID',
       'User', 'Submit', 'Eligible', 'Start', 'End', 'Elapsed',
       'ExitCode', 'State', 'NNodes', 'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES',
       'ReqTRES', 'Timelimit', 'NodeList', 'JobName']
accre = pd.read_csv('data/accre_data.txt', sep='|', usecols=required_cols)

C:\Users\Alex\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
accre.head()

,JobID,Partition,Account,Group,GID,User,Submit,Eligible,Start,End,...,State,NNodes,NCPUS,ReqCPUS,ReqMem,ReqGRES,ReqTRES,Timelimit,NodeList,JobName
0,15806649,production,h_vuiis,h_vuiis,36052,masispider,2017-07-02T00:40:58,2017-07-02T00:42:59,2017-07-03T02:13:59,2017-07-04T03:19:20,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1085,WOODWARD_TCP-x-228324-x-228324-x-301-x-Multi_A...
1,15806664,production,h_vuiis,h_vuiis,36052,masispider,2017-07-02T00:40:58,2017-07-02T00:42:59,2017-07-03T02:20:08,2017-07-04T02:11:55,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1099,WOODWARD_TCP-x-115174-x-115174-x-301-x-Multi_A...
2,15806670,production,h_vuiis,h_vuiis,36052,masispider,2017-07-02T00:40:58,2017-07-02T00:42:59,2017-07-03T02:41:13,2017-07-04T04:41:04,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp1126,WOODWARD_TCP-x-215607-x-215607-x-301-x-Multi_A...
3,15806671,production,h_vuiis,h_vuiis,36052,masispider,2017-07-02T00:40:58,2017-07-02T00:42:59,2017-07-03T02:51:53,2017-07-04T10:54:13,...,COMPLETED,1,2,2,12Gn,NaN,"cpu=2,mem=12G,node=1",9-04:00:00,vmp230,WOODWARD_TCP-x-116214-x-116214-x-1701-x-Multi_...
4,15811617,production,cms_samtest,cms_samtest,59297,uscms010,2017-07-01T05:00:00,2017-07-01T05:00:00,2017-07-01T05:00:01,2017-07-01T05:00:11,...,COMPLETED,1,1,1,4000Mn,NaN,"cpu=1,mem=4000M,node=1",2-00:00:00,vmp1242,bl_aa7c22e04f21


In [15]:
# look at the shape of the dataset
accre.shape

(12499389, 22)

In [16]:
# identify the column names of the dataset
accre.columns

Index(['JobID', 'Partition', 'Account', 'Group', 'GID', 'User', 'Submit',
       'Eligible', 'Start', 'End', 'Elapsed', 'ExitCode', 'State', 'NNodes',
       'NCPUS', 'ReqCPUS', 'ReqMem', 'ReqGRES', 'ReqTRES', 'Timelimit',
       'NodeList', 'JobName'],
      dtype='object')

In [19]:
# check data types
accre.JobID.value_counts(ascending=False).head(5000)

19224428_76       2
26828657_2        2
26764369_75       2
19224428_122      2
19224420_105      2
16905237          2
26764369_7        2
26828657_43       2
26828657_100      2
26764369_35       2
21748064          2
26827147_93       2
19224428_80       2
19224428_91       2
26827147_82       2
16891962_24       2
16912682          2
26747804_25       2
26749987_28       2
16891962_27       2
19224420_124      2
26763523_21       2
26782504          2
16891962_12       2
26828657_85       2
26828657_31       2
26827147_2        2
26747804_4        2
16905197_0        2
26827147_54       2
                 ..
16883505          1
27608659          1
18557606_5086     1
21854163          1
20718421          1
22505070          1
26313326_102      1
27516876_217      1
25612959          1
27688268          1
26929857_256      1
17931172          1
25875914_2        1
19996232          1
28109759_10108    1
22287256          1
21676503          1
27453431          1
24631188          1


In [14]:
# convert Submit and Start to datetime
accre['Submit'] = pd.to_datetime(accre['Submit'], format='%Y/%m/%d')
accre['Start'] = pd.to_datetime(accre['Start'], format='%Y/%m/%d')

In [15]:
# check head of Submit
accre.Submit.head()

0   2017-07-02 00:40:58
1   2017-07-02 00:40:58
2   2017-07-02 00:40:58
3   2017-07-02 00:40:58
4   2017-07-01 05:00:00
Name: Submit, dtype: datetime64[ns]

In [9]:
# check head of Start
accre.Start.head()

0   2017-07-03 02:13:59
1   2017-07-03 02:20:08
2   2017-07-03 02:41:13
3   2017-07-03 02:51:53
4   2017-07-01 05:00:01
Name: Start, dtype: datetime64[ns]

In [10]:
# create Queue_Time column = Start - Submit
accre['Queue_Time'] = accre['Start'] - accre['Submit']
accre.Queue_Time.head()

0   1 days 01:33:01
1   1 days 01:39:10
2   1 days 02:00:15
3   1 days 02:10:55
4   0 days 00:00:01
Name: Queue_Time, dtype: timedelta64[ns]